In [1]:
%pip install opencv-python-headless


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   -- ------------------------------------- 2.4/38.8 MB 11.2 MB/s eta 0:00:04
   --- ------------------------------------ 3.7/38.8 MB 8.1 MB/s eta 0:00:05
   ----- ---------------------------------- 5.0/38.8 MB 7.7 MB/s eta 0:00:05
   ----- ---------------------------------- 5.8/38.8 MB 7.0 MB/s eta 0:00:05
   ------ --------------------------------- 6.6/38.8 MB 5.9 MB/s eta 0:00:06
   ------- -------------------------------- 7.3/38.8 MB 5.5 MB/s eta 0:00:06
   -------- ------------------------------- 8.1/38.8 MB 5.4 MB/s eta 0:00:06
   --------- ------------------------------ 9.4/38.8 MB 5.2 MB/s eta 0:00:06
   ----------- ---------------------------- 10.7/38.8 MB 5.2 MB/s eta 0:00:06
   ------------ --------------------------- 12.1/38.8 MB 5.4 MB/s eta 0:00:05
   -------------- ------------------------- 14.4/38.8 MB 5.6 MB/s eta 0:00:05
   ---------------- ----------------------- 16.3/38.8 MB 5.8 MB/s eta 0:00:04


## Image Processing 

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:

image_path = 'data/ipl_img.png'
img = cv2.imread(image_path)

In [ ]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
kernel = np.array([[-2, -2, -2],
                   [-2, 16, -2],
                   [-2, -2, -2]])

In [ ]:
convolved_img = cv2.filter2D(gray_img, -1, kernel)

In [ ]:

plt.figure(figsize=(10, 5))

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(convolved_img, cmap='gray')
plt.title('Convolved Image (Edge Detection)')

plt.show()

## Semantic Analysis for IPL_Comments

In [ ]:
!pip install opencv-python-headless

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
comments_df = pd.read_csv('data/ipl_comments.csv')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

comments_df['cleaned_comment'] = comments_df['comment'].apply(preprocess_text)

In [ ]:
tfidf = TfidfVectorizer(max_features=500)
X = tfidf.fit_transform(comments_df['cleaned_comment']).toarray()

In [ ]:
y = comments_df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_prob = model.predict_proba(X)
y_pred = model.predict(X)

In [ ]:
for i in range(len(comments_df)):
    comment = comments_df['comment'][i]
    predicted_sentiment = y_pred[i]
    prob_negative = y_prob[i][0]
    prob_positive = y_prob[i][1]
    
    print(f"Sentence: {comment}")
    print(f"Predicted Sentiment: {predicted_sentiment} (Probabilities: Negative: {prob_negative:.4f}, Positive: {prob_positive:.4f})")
    print("-" * 50)